<div align="center">

# Assignment 11

---

**Author:** Matthew Walczyk  
**Date:** 10/28/2024  
**Modified By:** Matthew Walczyk  
**Description:** This program contains my week 11 exercises using sqlite3 pandas and sqlalchemy soup.

---
</div>

In [20]:
import sqlite3
import pandas as pd
import random
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey
from sqlalchemy.orm import declarative_base, sessionmaker, relationship

In [21]:
conn = sqlite3.connect(':memory:')  # Use an in-memory database
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE sensors (
    id INTEGER PRIMARY KEY,
    sensor_type TEXT NOT NULL,
    location TEXT,
    reading FLOAT,
    year INTEGER,
    sunactivity INTEGER
)
''')

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables in the database:", cursor.fetchall())

for _ in range(5):
    cursor.execute('''
        INSERT INTO sensors (sensor_type, location, reading, year, sunactivity)
        VALUES (?, ?, ?, ?, ?)
    ''', (
        random.choice(['Temperature', 'Humidity', 'Pressure']),
        random.choice(['North', 'South', 'East', 'West']),
        random.uniform(0, 100),
        random.randint(1700, 2023),
        random.randint(0, 30)
    ))

cursor.execute("SELECT * FROM sensors")
print("Data in sensors table:", cursor.fetchall())

cursor.execute("DROP TABLE sensors")
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables after dropping sensors:", cursor.fetchall())

conn.close()

Tables in the database: [('sensors',)]
Data in sensors table: [(1, 'Humidity', 'North', 45.39154559391658, 1741, 26), (2, 'Humidity', 'North', 10.35625498123709, 2014, 15), (3, 'Humidity', 'West', 25.95534116633871, 1880, 13), (4, 'Pressure', 'West', 92.11058049764453, 2001, 16), (5, 'Humidity', 'West', 19.264695914746767, 1723, 0)]
Tables after dropping sensors: []


In [22]:
data = [
    ('Temperature', 'North', 25.3, 1725, 15),
    ('Humidity', 'South', 50.8, 1800, 22),
    ('Pressure', 'East', 75.5, 1750, 18),
    ('Temperature', 'West', 85.6, 1690, 5),
    ('Humidity', 'North', 65.4, 1720, 10)
]
df = pd.DataFrame(data, columns=['sensor_type', 'location', 'reading', 'year', 'sunactivity'])

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
cursor.execute('CREATE TABLE sensors (sensor_type, location, reading, year, sunactivity)')

df.to_sql('sensors', conn, if_exists='append', index=False)

cursor.execute("SELECT COUNT(*) FROM sensors")
print("Row count after insert:", cursor.fetchone()[0])

cursor.execute("DELETE FROM sensors WHERE sunactivity > 20")
conn.commit()

cursor.execute("SELECT * FROM sensors WHERE year < 1732")
print("Records with year < 1732:", cursor.fetchall())

conn.close()

Row count after insert: 5
Records with year < 1732: [('Temperature', 'North', 25.3, 1725, 15), ('Temperature', 'West', 85.6, 1690, 5), ('Humidity', 'North', 65.4, 1720, 10)]


In [23]:
Base = declarative_base()

class Station(Base):
    __tablename__ = 'stations'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    location = Column(String)
    sensors = relationship("Sensor", back_populates="station")

class Sensor(Base):
    __tablename__ = 'sensors'
    
    id = Column(Integer, primary_key=True)
    sensor_type = Column(String, nullable=False)
    reading = Column(Float)
    station_id = Column(Integer, ForeignKey('stations.id'))
    station = relationship("Station", back_populates="sensors")

engine = create_engine('sqlite:///:memory:')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

station1 = Station(name='Station1', location='North')
station2 = Station(name='Station2', location='South')
session.add_all([station1, station2])
session.commit()

sensor1 = Sensor(sensor_type='Temperature', reading=23.5, station=station1)
session.add(sensor1)
session.commit()

session.close()

In [24]:
engine = create_engine('sqlite:///example.db')

Base = declarative_base()

class Station(Base):
    __tablename__ = 'stations'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    location = Column(String)
    
    sensors = relationship("Sensor", back_populates="station")

class Sensor(Base):
    __tablename__ = 'sensors'
    
    id = Column(Integer, primary_key=True)
    sensor_type = Column(String, nullable=False)
    reading = Column(Float)
    station_id = Column(Integer, ForeignKey('stations.id'))
    
    station = relationship("Station", back_populates="sensors")

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

# Add two stations
station1 = Station(name='Station1', location='North')
station2 = Station(name='Station2', location='South')
session.add_all([station1, station2])
session.commit()

# Add sensors linked to these stations
sensor1 = Sensor(sensor_type='Temperature', reading=23.5, station=station1)
sensor2 = Sensor(sensor_type='Humidity', reading=45.7, station=station1)
sensor3 = Sensor(sensor_type='Pressure', reading=1013.2, station=station2)
session.add_all([sensor1, sensor2, sensor3])
session.commit()

first_station = session.query(Station).first()
print("First station:", first_station.name, first_station.location)

all_stations = session.query(Station).all()
print("All stations:", [(station.name, station.location) for station in all_stations])

all_sensors = session.query(Sensor).all()
print("All sensors:", [(sensor.sensor_type, sensor.reading) for sensor in all_sensors])

first_sensor = session.query(Sensor).filter(Sensor.station_id == first_station.id).first()
if first_sensor:
    print("First sensor in first station:", first_sensor.sensor_type, first_sensor.reading)
else:
    print("No sensor found for the first station.")

df_stations = pd.read_sql("stations", con=engine)
print("Stations DataFrame:\n", df_stations)

session.close()

First station: Station1 North
All stations: [('Station1', 'North'), ('Station2', 'South'), ('Station1', 'North'), ('Station2', 'South'), ('Station1', 'North'), ('Station2', 'South'), ('Station1', 'North'), ('Station2', 'South'), ('Station1', 'North'), ('Station2', 'South')]
All sensors: [('Temperature', 23.5), ('Humidity', 45.7), ('Pressure', 1013.2), ('Temperature', 23.5), ('Humidity', 45.7), ('Pressure', 1013.2), ('Temperature', 23.5), ('Humidity', 45.7), ('Pressure', 1013.2), ('Temperature', 23.5), ('Humidity', 45.7), ('Pressure', 1013.2), ('Temperature', 23.5), ('Humidity', 45.7), ('Pressure', 1013.2)]
First sensor in first station: Temperature 23.5
Stations DataFrame:
    id      name location
0   1  Station1    North
1   2  Station2    South
2   3  Station1    North
3   4  Station2    South
4   5  Station1    North
5   6  Station2    South
6   7  Station1    North
7   8  Station2    South
8   9  Station1    North
9  10  Station2    South
